# **Recommendation system-**

Collaborative Filtering

In [13]:
import pandas as pd
import numpy as np

master_df = pd.read_csv("../data/processed/master_cleaned.csv")

master_df.head()

,TransactionId,UserId,VisitYear,VisitMonth,AttractionId,Rating,VisitMode,ContinentId,RegionId,CountryId,CityId,CityName,Country,Region,Continent,AttractionCityId,AttractionType,Attraction,AttractionAddress
0,3,70456,2022,10,640,5,Couples,5.0,21.0,163.0,4341.0,Guildford,United Kingdom,Western Europe,Europe,1,Nature & Wildlife Areas,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia"
1,8,7567,2022,10,640,5,Friends,2.0,8.0,48.0,464.0,Ontario,Canada,Northern America,America,1,Nature & Wildlife Areas,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia"
2,9,79069,2022,10,640,5,Family,2.0,9.0,54.0,774.0,Brazil,Brazil,South America,America,1,Nature & Wildlife Areas,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia"
3,10,31019,2022,10,640,3,Family,5.0,17.0,135.0,583.0,Zurich,Switzerland,Central Europe,Europe,1,Nature & Wildlife Areas,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia"
4,15,43611,2022,10,640,3,Couples,5.0,21.0,163.0,1396.0,Manchester,United Kingdom,Western Europe,Europe,1,Nature & Wildlife Areas,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia"


In [14]:
user_item_matrix = master_df.pivot_table(
    index='UserId',
    columns='AttractionId',
    values='Rating'
)

user_item_matrix.head()

AttractionId,369,481,640,650,673,737,748,749,824,841,...,1133,1137,1166,1171,1220,1225,1238,1278,1280,1297
UserId,,,,,,,,,,,,,,,,,,,,,
14,NaN,NaN,4.00,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,5.0,4.25,NaN,NaN,NaN,5.0,NaN,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
user_item_matrix = user_item_matrix.fillna(0)

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_matrix)

user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

user_similarity_df.head()

UserId,14,16,20,23,25,26,27,28,29,32,...,88168,88170,88175,88177,88178,88179,88185,88187,88189,88190
UserId,,,,,,,,,,,,,,,,,,,,,
14,1.000000,0.603672,0.000000,0.780869,0.0,0.780869,0.624695,0.0,0.624695,0.624695,...,0.624695,0.404029,0.624695,0.624695,0.624695,0.780869,0.327429,0.000000,0.499756,0.780869
16,0.603672,1.000000,0.460165,0.460165,0.0,0.460165,0.391141,0.0,0.391141,0.391141,...,0.391141,0.252975,0.391141,0.391141,0.391141,0.460165,0.578861,0.460165,0.312913,0.460165
20,0.000000,0.460165,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.314485,0.000000,0.000000,0.000000
23,0.780869,0.460165,0.000000,1.000000,0.0,1.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.419314,0.000000,0.000000,1.000000
25,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
def recommend_attractions(user_id, top_n=5):

    # Get similarity scores for this user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)

    # Remove self
    similar_users = similar_users.drop(user_id)

    # Get top similar users
    top_users = similar_users.head(10).index

    # Get their ratings
    similar_users_ratings = user_item_matrix.loc[top_users]

    # Average rating for each attraction
    recommendation_scores = similar_users_ratings.mean(axis=0)

    # Remove already visited attractions
    user_rated = user_item_matrix.loc[user_id]
    recommendation_scores = recommendation_scores[user_rated == 0]

    # Return top recommendations
    top_recommendations = recommendation_scores.sort_values(ascending=False).head(top_n)

    return top_recommendations

In [18]:
sample_user = user_item_matrix.index[0]

recommend_attractions(sample_user)

AttractionId
369    0.0
481    0.0
650    0.0
673    0.0
737    0.0
dtype: float64

In [19]:
def recommend_with_names(user_id, top_n=5):

    rec_ids = recommend_attractions(user_id, top_n).index

    recommendations = master_df[
        master_df['AttractionId'].isin(rec_ids)
    ][['AttractionId', 'Attraction', 'AttractionType']].drop_duplicates()

    return recommendations

recommend_with_names(sample_user)

,AttractionId,Attraction,AttractionType
19618,673,Seminyak Beach,Beaches
22528,481,Nusa Dua Beach,Beaches
24631,650,Sanur Beach,Beaches
39024,737,Tanah Lot Temple,Religious Sites
42374,369,Kuta Beach - Bali,Beaches


In [20]:
# Train-Test Split
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    master_df,
    test_size=0.2,
    random_state=42
)

In [21]:
# Build Train Matrix
train_matrix = train_df.pivot_table(
    index='UserId',
    columns='AttractionId',
    values='Rating'
).fillna(0)

In [22]:
# Compute Similarity on Train
from sklearn.metrics.pairwise import cosine_similarity

train_similarity = cosine_similarity(train_matrix)

train_similarity_df = pd.DataFrame(
    train_similarity,
    index=train_matrix.index,
    columns=train_matrix.index
)

In [23]:
def predict_rating(user_id, attraction_id, k=10):

    if user_id not in train_matrix.index:
        return 0

    if attraction_id not in train_matrix.columns:
        return 0

    # Get similarity scores
    sim_scores = train_similarity_df[user_id]

    # Get users who rated this attraction
    attraction_ratings = train_matrix[attraction_id]
    rated_users = attraction_ratings[attraction_ratings > 0]

    # Keep only similar users who rated this attraction
    sim_scores = sim_scores[rated_users.index]

    # Select top K similar users
    top_users = sim_scores.sort_values(ascending=False).head(k)

    if top_users.sum() == 0:
        return 0

    weighted_sum = (top_users * rated_users[top_users.index]).sum()

    return weighted_sum / top_users.sum()

In [24]:
# Calculate RMSE
from sklearn.metrics import mean_squared_error
import numpy as np

predictions = []
actuals = []

for row in test_df.itertuples():

    if row.UserId in train_matrix.index and row.AttractionId in train_matrix.columns:

        pred = predict_rating(row.UserId, row.AttractionId)

        predictions.append(pred)
        actuals.append(row.Rating)

rmse = np.sqrt(mean_squared_error(actuals, predictions))

print("Recommendation RMSE:", rmse)

Recommendation RMSE: 1.6243517632121682


Content-Based Filtering

In [48]:
# Ensure one row per AttractionId
attraction_df = master_df.groupby('AttractionId').first().reset_index()

# Keep only needed columns
attraction_df = attraction_df[
    ['AttractionId', 'Attraction', 'AttractionType',
     'Country', 'Region', 'Continent']
]

# Confirm uniqueness
print("Unique AttractionIds:", attraction_df['AttractionId'].nunique())
print("Total rows:", len(attraction_df))

Unique AttractionIds: 30
Total rows: 30


In [49]:
attraction_features = pd.get_dummies(
    attraction_df[['AttractionType', 'Country', 'Region', 'Continent']],
    drop_first=True
)

attraction_features.index = attraction_df['AttractionId']

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

attraction_similarity = cosine_similarity(attraction_features)

attraction_similarity_df = pd.DataFrame(
    attraction_similarity,
    index=attraction_features.index,
    columns=attraction_features.index
)

In [51]:
def content_recommend(user_id, top_n=5):

    user_data = master_df[master_df['UserId'] == user_id]

    liked = user_data[user_data['Rating'] >= 4]['AttractionId']

    if len(liked) == 0:
        return "User has no highly rated attractions."

    # Initialize similarity scores as float Series
    similarity_scores = pd.Series(
        0.0,
        index=attraction_similarity_df.index
    )

    for attraction_id in liked:

        if attraction_id in attraction_similarity_df.index:

            sim_vector = attraction_similarity_df.loc[attraction_id]

            # Force to Series (in case pandas returns DataFrame)
            if isinstance(sim_vector, pd.DataFrame):
                sim_vector = sim_vector.iloc[0]

            similarity_scores += sim_vector

    # Remove visited
    visited = user_data['AttractionId']
    similarity_scores = similarity_scores.drop(visited, errors='ignore')

    # Now similarity_scores is guaranteed Series
    similarity_scores = similarity_scores.astype(float)

    top_ids = similarity_scores.sort_values(
        ascending=False
    ).head(top_n).index

    return attraction_df[
        attraction_df['AttractionId'].isin(top_ids)
    ][['Attraction', 'AttractionType', 'Country']]

In [52]:
sample_user = master_df['UserId'].iloc[0]

content_recommend(sample_user)

,Attraction,AttractionType,Country
1,Nusa Dua Beach,Beaches,United Kingdom
5,Tanah Lot Temple,Religious Sites,United Kingdom
6,Tegalalang Rice Terrace,Points Of Interest & Landmarks,United Kingdom
7,Tegenungan Waterfall,Waterfalls,United Kingdom
23,Merapi Volcano,Volcanos,United Kingdom


In [56]:
def content_predict_rating(user_id, attraction_id):

    user_data = train_df[train_df['UserId'] == user_id]

    rated_items = user_data[user_data['Rating'] > 0][['AttractionId', 'Rating']]

    if len(rated_items) == 0:
        return 0.0

    numerator = 0.0
    denominator = 0.0

    for row in rated_items.itertuples():

        rated_id = row.AttractionId
        rating = row.Rating

        if rated_id in attraction_similarity_df.index and \
           attraction_id in attraction_similarity_df.columns:

            sim_value = attraction_similarity_df.loc[rated_id, attraction_id]

            numerator += sim_value * rating
            denominator += abs(sim_value)

    if denominator == 0:
        return 0.0

    return numerator / denominator

In [57]:
from sklearn.metrics import mean_squared_error
import numpy as np

predictions = []
actuals = []

for row in test_df.itertuples():

    if row.AttractionId in attraction_similarity_df.columns:

        pred = content_predict_rating(row.UserId, row.AttractionId)

        predictions.append(pred)
        actuals.append(row.Rating)

rmse_content = np.sqrt(mean_squared_error(actuals, predictions))

print("Content-Based RMSE:", rmse_content)

Content-Based RMSE: 3.355037427618199
